In [1]:
import pandas as pd
import numpy as np

# NER
import spacy
import fr_core_news_md

### Import data

In [22]:
# import with features
df = pd.read_csv("../data/corpus_train_features.csv", index_col=0)
df_test = pd.read_csv("../data/corpus_test.csv", sep='|')

In [3]:
df.head()

,paragraph,author,len_paragraph,nb_tokens,nb_sentences,nb_tokens_per_sentences,nb_words,nb_unique_words,nb_punct,nb_punct_relative
0,Beaucoup de gens ont envié la douce existence ...,Balzac,1027,192,5,37.400000,172,123,20,0.104167
1,"Un mercredi, de grand matin, vers le milieu du...",Balzac,425,85,2,41.500000,73,63,12,0.141176
2,"--Ah! c'est toi, Suzanne? dit le chevalier de ...",Balzac,211,41,4,9.250000,35,33,6,0.146341
3,"Cette charmante Suzanne, dont la comique avent...",Balzac,5557,1015,33,29.757576,887,439,128,0.126108
4,"Suzanne, une de ses favorites, spirituelle, am...",Balzac,563,107,2,52.500000,91,77,16,0.149533


In [4]:
df.shape

(47202, 10)

### Named entities recognition

In [5]:
nlp = fr_core_news_md.load()

In [6]:
doc = nlp("Margot habite à Paris, capitale de la France, où il pleut tout le temps")

for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)
    
    
doc = nlp("Victor Hugo et Honoré de Balzac sont des grands écrivains du 19ème siècle")

for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

Margot 0 6 PER
Paris 16 21 LOC
Victor Hugo 0 11 PER
Honoré de Balzac 15 31 PER


#### Fonction that remove the named entities from a paragraph.

In [14]:
def remove_named_entities(paragraph):
    #nlp = fr_core_news_md.load()
    doc = nlp(paragraph)
    
    names = []
    for ent in doc.ents:
        # list of all the named entities recognized in the paragraph
        names.append(ent)
    # keep unique elements and convert to string    
    names = set([str(x) for x in names])
    
    # remove named entities from the paragraph
    paragraph_no_names = paragraph
    for name in names:
        paragraph_no_names = paragraph_no_names.replace(name, '')
        
    return paragraph_no_names

In [15]:
par_test = df['paragraph'][0]

In [16]:
par_test

"Beaucoup de gens ont envié la douce existence de ce vieux garçon, pleine de parties de boston, de trictrac, de reversi, de whist et de piquet bien jouées, de dîners bien digérés, de prises de tabac humées avec grâce, de tranquilles promenades. Presque tout Alençon croyait cette vie exempte d'ambition et d'intérêts graves; mais aucun homme n'a une vie aussi simple que ses envieux la lui font. Vous découvrirez dans les villages les plus oubliés des mollusques humains, des rotifères en apparence morts, qui ont la passion des lépidoptères ou de la conchyliologie, et qui se donnent des maux infinis pour je ne sais quels papillons ou pour la _concha Veneris_. Non-seulement le chevalier avait ses coquillages, mais encore il nourrissait un ambitieux désir poursuivi avec une profondeur digne de Sixte-Quint: il voulait se marier avec une vieille fille riche, sans doute dans l'intention de s'en faire un marchepied pour aborder les sphères élevées de la cour. Là était le secret de sa royale tenue

In [17]:
doc_test = nlp(par_test)

for ent in doc_test.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

Alençon 257 264 LOC
Veneris_. 652 661 LOC
Sixte 797 802 PER
Quint 803 808 PER
Alençon 1019 1026 LOC


In [18]:
remove_named_entities(par_test)

"Beaucoup de gens ont envié la douce existence de ce vieux garçon, pleine de parties de boston, de trictrac, de reversi, de whist et de piquet bien jouées, de dîners bien digérés, de prises de tabac humées avec grâce, de tranquilles promenades. Presque tout  croyait cette vie exempte d'ambition et d'intérêts graves; mais aucun homme n'a une vie aussi simple que ses envieux la lui font. Vous découvrirez dans les villages les plus oubliés des mollusques humains, des rotifères en apparence morts, qui ont la passion des lépidoptères ou de la conchyliologie, et qui se donnent des maux infinis pour je ne sais quels papillons ou pour la _concha  Non-seulement le chevalier avait ses coquillages, mais encore il nourrissait un ambitieux désir poursuivi avec une profondeur digne de -: il voulait se marier avec une vieille fille riche, sans doute dans l'intention de s'en faire un marchepied pour aborder les sphères élevées de la cour. Là était le secret de sa royale tenue et de son séjour à ."

#### Create new column with the paragraphs without the named entities.

Apply this to train and test sets.

In [19]:
paragraph_ner = []
for i, par in enumerate(df['paragraph']):
    par_ner = remove_named_entities(par)
    paragraph_ner.append(par_ner)
    if i%1000 == 0:
        print(i)

df['paragraph_ner'] = paragraph_ner

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000


In [23]:
paragraph_test_ner = []
for i, par in enumerate(df_test['paragraph']):
    par_ner = remove_named_entities(par)
    paragraph_test_ner.append(par_ner)
    if i%1000 == 0:
        print(i)

df_test['paragraph_ner'] = paragraph_test_ner

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000


#### Export new train file with all the features

In [20]:
# save complete train dataframe to csv
df.to_csv('../data/corpus_train_features_NER.csv')

In [24]:
# save complete test dataframe to csv
df_test.to_csv('../data/corpus_test_NER.csv')

#### Check problems with NER

In [40]:
# voir pb pour ligne 4 "Suzanne" encore present apres NER?
print(df['paragraph'][4])
remove_named_entities(df['paragraph'][4])

Suzanne, une de ses favorites, spirituelle, ambitieuse, avait en elle l'étoffe d'une Sophie Arnould, elle était d'ailleurs belle comme la plus belle courtisane que jamais Titien ait conviée à poser sur un velours noir pour aider son pinceau à faire une Vénus; mais sa figure, quoique fine dans le tour des yeux et du front, péchait en bas par des contours communs. C'était la beauté normande, fraîche, éclatante, rebondie, la chair de Rubens qu'il faudrait marier avec les muscles de l'Hercule-Farnèse, et non la Vénus de Médicis, cette gracieuse femme d'Apollon.


"Suzanne, une de ses favorites, spirituelle, ambitieuse, avait en elle l'étoffe d'une , elle était d'ailleurs belle comme la plus belle courtisane que jamais  ait conviée à poser sur un velours noir pour aider son pinceau à faire une ; mais sa figure, quoique fine dans le tour des yeux et du front, péchait en bas par des contours communs. C'était la beauté normande, fraîche, éclatante, rebondie, la chair de  qu'il faudrait marier avec les muscles de l'-, et non la  de , cette gracieuse femme d'."

In [29]:
print(df['paragraph'][30])
remove_named_entities(df['paragraph'][30])

--Suzanne, que diable veux-tu que je fasse? il est si extraordinaire.... Moi qui croyais... Le fait est que... mais non, non, cela ne se peut pas...


'--Suzanne, que diable veux-tu que je fasse? il est si extraordinaire.... Moi qui croyais... Le fait est que... mais non, non, cela ne se peut pas...'

In [42]:
doc = nlp(df['paragraph'][4])

for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

Sophie Arnould 85 99 PER
Titien 171 177 PER
Vénus 253 258 LOC
Rubens 435 441 PER
Hercule 486 493 LOC
Farnèse 494 501 PER
Vénus 513 518 LOC
Médicis 522 529 PER
Apollon 555 562 PER


In [43]:
doc = nlp(df['paragraph'][30])

for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)